#### Load vào bảng DIM MOVIE

In [0]:
%sql
CREATE OR REPLACE TABLE dim_movie_stage_1 AS
SELECT
  DISTINCT title,
  release_date,
  overview
FROM transformdataallcollect
WHERE time_updated > (
  SELECT IFNULL(MAX(time_updated), TO_DATE('1900-01-01'))
  FROM dim_movie
)

In [0]:
%sql
CREATE OR REPLACE TABLE dim_movie_stage_2 AS
SELECT
  CASE WHEN goldDim.title IS NULL
    THEN row_number() OVER (ORDER BY silverDim.title)
    ELSE goldDim.movie_id
  END AS movie_id,
  silverDim.title,
  silverDim.release_date,
  silverDim.overview,
  goldDim.title AS gold_title
FROM dim_movie_stage_1 silverDim
LEFT OUTER JOIN dim_movie goldDim
ON silverDim.title = goldDim.title
WHERE goldDim.title IS NULL
  OR silverDim.release_date <> goldDim.release_date
  OR silverDim.overview <> goldDim.overview

In [0]:
%sql
CREATE OR REPLACE TABLE dim_movie_stage_3 AS
SELECT  
  CASE WHEN gold_title IS NULL
    THEN movie_id + MAX_MOVIE
    ELSE movie_id
  END AS movie_id,
  title,
  release_date,
  overview
FROM dim_movie_stage_2
CROSS JOIN (
  SELECT IFNULL(MAX(movie_id), 0) AS MAX_MOVIE
  FROM dim_movie
)

In [0]:
%sql
MERGE INTO dim_movie goldDim
USING dim_movie_stage_3 silverDim
ON goldDim.title = silverDim.title
WHEN MATCHED THEN
  UPDATE SET
    goldDim.release_date = silverDim.release_date,
    goldDim.overview = silverDim.overview,
    goldDim.time_updated = current_timestamp()
WHEN NOT MATCHED THEN
  INSERT (
    movie_id,
    title,
    release_date,
    overview,
    time_inserted,
    time_updated
    )
    VALUES (
      silverDim.movie_id,
      silverDim.title,
      silverDim.release_date,
      silverDim.overview,
      current_timestamp(),
      current_timestamp()
      )

#### Load vào bảng DIM COMPANY


In [0]:
%sql
CREATE OR REPLACE TABLE dim_company_stage_1 AS
SELECT
  DISTINCT company,
  country
FROM transformdataallcollect
WHERE time_updated > (
  SELECT IFNULL(MAX(time_updated), TO_DATE('1900-01-01'))
  FROM dim_company
)

In [0]:
%sql
CREATE OR REPLACE TABLE dim_company_stage_2 AS
SELECT
  CASE WHEN goldDim.company_name IS NULL
    THEN row_number() OVER (ORDER BY silverDim.company)
    ELSE goldDim.company_id
  END AS company_id,
  silverDim.company,
  silverDim.country,
  goldDim.company_name AS gold_company
FROM dim_company_stage_1 silverDim
LEFT OUTER JOIN dim_company goldDim
ON silverDim.company = goldDim.company_name
WHERE goldDim.company_name IS NULL
  OR silverDim.country <> goldDim.country

In [0]:
%sql
CREATE OR REPLACE TABLE dim_company_stage_3 AS
SELECT  
  CASE WHEN gold_company IS NULL
    THEN company_id + MAX_COMPANY
    ELSE company_id
  END AS company_id,
  company,
  country
FROM dim_company_stage_2
CROSS JOIN (
  SELECT IFNULL(MAX(company_id), 0) AS MAX_COMPANY
  FROM dim_company
)

In [0]:
%sql
MERGE INTO dim_company goldDim
USING dim_company_stage_3 silverDim
ON goldDim.company_name = silverDim.company
WHEN MATCHED THEN
  UPDATE SET
    goldDim.country = silverDim.country,
    goldDim.time_updated = current_timestamp()
WHEN NOT MATCHED THEN
  INSERT (
    company_id,
    company_name,
    country,
    time_inserted,
    time_updated
    )
  VALUES (
    silverDim.company_id,
    silverDim.company,
    silverDim.country,
    current_timestamp(),
    current_timestamp()
    )
  

#### Load vào bảng DIM DIRECTOR


In [0]:
%sql
CREATE OR REPLACE TABLE dim_director_stage_1 AS
SELECT
  DISTINCT directors
FROM transformdataallcollect
WHERE time_updated > (
  SELECT IFNULL(MAX(time_updated), TO_DATE('1900-01-01'))
  FROM dim_director
)

In [0]:
%sql
CREATE OR REPLACE TABLE dim_director_stage_2 AS
SELECT
  row_number() OVER (ORDER BY silverDim.directors) AS director_id,
  silverDim.directors
FROM dim_director_stage_1 silverDim
LEFT OUTER JOIN dim_director goldDim
ON silverDim.directors = goldDim.director_name
WHERE goldDim.director_name IS NULL

In [0]:
%sql
CREATE OR REPLACE TABLE dim_director_stage_3 AS
SELECT  
  director_id + MAX_DIRECTOR AS director_id,
  directors
FROM dim_director_stage_2
CROSS JOIN (
  SELECT IFNULL(MAX(director_id), 0) AS MAX_DIRECTOR
  FROM dim_director
)

In [0]:
%sql
INSERT INTO dim_director
SELECT
  director_id,
  directors,
  current_timestamp() AS time_inserted,
  current_timestamp() AS time_updated
FROM dim_director_stage_3